In [7]:
import pprint
import random
import copy

In [8]:
from dataclasses import dataclass

@dataclass
class Link:
    start_node: int 
    end_node: int
    number_of_modules: int
    module_cost: int
    link_module: int

@dataclass
class Demand:
    start_node: int
    end_node: int
    demand_volume: int
    number_of_paths: int
    paths: list
    
@dataclass
class Path:
    demand_path_id: int
    links: list
    
@dataclass
class Network:
    number_of_links: int
    links: list
    number_of_demands: int
    demands: list

In [9]:
def load_network(network_file):
    lines = []

    with open(network_file) as f:
        for line in f:
            lines.append(line[:-1])

    number_of_links = int(lines[0])

    links = []

    for i in range(1, number_of_links + 1):
        start_node, end_node, number_of_modules, module_cost, link_module = [int(item) for item in lines[i].split()]
        
        links.append(Link(start_node, end_node, number_of_modules, module_cost, link_module))

    number_of_demands = int(lines[number_of_links + 3])

    index_line = number_of_links + 5

    links_path = []
    demands = []

    for i in range(number_of_demands):
        paths = []

        start_node, end_node, demand_volume = [int(item) for item in lines[index_line].split()]
        
        number_of_paths = int(lines[index_line + 1])

        for j in range(index_line + 2, index_line + number_of_paths + 2):
            current_line = [int(item) for item in lines[j].split()]
            
            demand_path_id, links_path = current_line[0], current_line[1:]
            
            paths.append(Path(demand_path_id, links_path))

        demands.append(Demand(start_node, end_node, demand_volume, number_of_paths, paths))    

        index_line += number_of_paths + 3
    
    return Network(number_of_links, links, number_of_demands, demands)

In [228]:
from abc import ABC, abstractmethod

class Individual(ABC):
    def __init__(self, value=None, init_params=None):
        if value is not None:
            self.value = value
        else:
            self.value = self._random_init(init_params)

    @abstractmethod
    def pair(self, other, pair_params):
        pass

    @abstractmethod
    def mutate(self, mutate_params):
        pass

    @abstractmethod
    def _random_init(self, init_params):
        pass
    
class Fitness():
    @staticmethod
    def fitnessDAP(solution):
        print("Calculating fitness function DAP")
    
        flows_allocation = 0
        max_load = 0
    
        for link in range(1, network.number_of_links + 1):
            link_load = 0
            for demand in range(0, network.number_of_demands):
                for path in network.demands[demand].paths:
                    if link in path.links:
                        link_load += solution.value[demand][path.demand_path_id - 1]
            if link_load > max_load:
                max_load = link_load
    
        print(max_load)
        return max_load  
        
    @staticmethod    
    def fitnessDDAP(solution):
        print("Calculating fitness function DDAP")
    
        flows_allocation = 0
    
        for link in range(1, network.number_of_links + 1):
            link_load = 0
            for demand in range(0, network.number_of_demands):
                for path in network.demands[demand].paths:
                    if link in path.links:
                        link_load += solution.value[demand][path.demand_path_id - 1]
            link_size = link_load / network.links[link - 1].link_module
            flows_allocation += network.links[link - 1].module_cost * link_size
    
        print(flows_allocation)
        return flows_allocation  

In [290]:
class Solution(Individual):
    def pair(self, other, pair_params):
        print("Pairing")
        pairing_probability = pair_params["pairing_probability"]
        #random.seed(pair_params["seed"])
        flows = []
        
        if pairing_probability > random.uniform(0, 1):
            for current_demand in range(pair_params["number_of_demands"]):
                if random.choice([0, 1]) == 0:
                    flows.append(self.value[current_demand])
                else:
                    flows.append(other.value[current_demand])
        
        print("Child " + str(flows))
        
        return Solution(flows)
        
    
    def child_complement(self, mother, father):
        child = []
        
        for i in range(len(self.value)):
            if self.value[i] == mother.value[i]:
                child.append(father.value[i])
            else:
                child.append(mother.value[i])
        
        print("Child " + str(child))
        
        return Solution(child)
            
    
    def mutate(self, mutate_params):
        print("Mutation")
        mutation_probability = mutate_params["mutation_probability"]
        #random.seed(mutate_params["seed"])
        
        if mutation_probability > random.uniform(0, 1):
            selected_demand = random.randint(0, mutate_params["number_of_demands"] - 1)
            print("Selected demand " + str(selected_demand + 1))
            
            if len(self.value[selected_demand]) == 1:
                print("Only one demand unit, canceling mutation")
                return
            
            demand_units = copy.copy(self.value[selected_demand])
            print("Demand units " + str(demand_units))
            
            positive_demand_units = [i for i, e in enumerate(demand_units) if e != 0]
            
            selected_path = random.choice(positive_demand_units)
            print("Selected demand in path to shift from " + str(demand_units[selected_path]))
            
            not_selected_paths = [i for i, e in enumerate(demand_units) if i != selected_path]
            
            selected_path_to_shift = random.choice(not_selected_paths)
            print("Selected demand in path to shift to " + str(demand_units[selected_path_to_shift]))

            demand_units[selected_path] -= 1
            demand_units[selected_path_to_shift] += 1
            self.value[selected_demand] = demand_units
            
            print("Child after mutation \n" + str(self.value))
        else:
            print("No mutation")

    def _random_init(self, init_params):
        print("Individual initialization")
        #random.seed(init_params["seed"])
        
        flows = []
    
        for number_of_paths, demand_volume in zip(init_params["number_of_paths_list"], init_params["demand_volume_list"]):
            demand_units = []
            demand_limit = copy.copy(demand_volume)

            for i in range(number_of_paths - 1):
                current_units = random.randint(0, demand_limit)
                demand_units.append(current_units)
                demand_limit -= current_units

            demand_units.append(demand_limit)
            flows.append(demand_units)
        
        print(flows)
        
        return flows
    
class Population:
    def __init__(self, size, fitness, individual_class, init_params):
        print("Population initialization")
        self.fitness = fitness
        self.individuals = [individual_class(init_params=init_params) for _ in range(size)]
        self.individuals.sort(key=lambda x: self.fitness(x))

    def replace(self, new_individuals):
        size = len(self.individuals)
        self.individuals.extend(new_individuals)
        
        self.individuals.sort(key=lambda x: self.fitness(x))
        self.individuals = self.individuals[:size]
        
        print("Take best individuals")
        for individual in self.individuals:
            print(individual.value)

    def get_parents(self, n_offsprings):
        print("Get parents of individuals")
        mothers = self.individuals[-2 * n_offsprings::2]
        fathers = self.individuals[-2 * n_offsprings + 1::2]
        
        for mother in mothers:
            print("mother " + str(mother.value))
            
        for father in fathers:
            print("father " + str(father.value))
        
        return mothers, fathers
    

class Evolution:
    def __init__(self, pool_size, fitness, individual_class, n_offsprings, pair_params, mutate_params, init_params):
        print("Evolution initialization")
        self.pair_params = pair_params
        self.mutate_params = mutate_params
        self.pool = Population(pool_size, fitness, individual_class, init_params)
        self.n_offsprings = n_offsprings

    def step(self):
        print("Step of evolution")
        mothers, fathers = self.pool.get_parents(self.n_offsprings)
        offsprings = []

        for mother, father in zip(mothers, fathers):
            offspring_first = mother.pair(father, self.pair_params)
            if not offspring_first.value:
                print("No pairing")
            else:
                offspring_second = offspring_first.child_complement(mother, father)
            
                offspring_second.mutate(self.mutate_params)
            
                offsprings.append(offspring_first)
                offsprings.append(offspring_second)

        self.pool.replace(offsprings)

In [288]:
network = load_network("net12_1.txt")

In [291]:
random.seed(36)

evo = Evolution(
    pool_size=8, fitness=Fitness.fitnessDAP, individual_class=Solution, n_offsprings=2,
    pair_params = {"seed": 55, "number_of_demands": network.number_of_demands, "pairing_probability": 0.9},
    mutate_params = {"seed": 19, "number_of_demands": network.number_of_demands, "mutation_probability": 0.9},
    init_params = {"seed": 22, "number_of_paths_list": [x.number_of_paths for x in network.demands], "demand_volume_list": [x.demand_volume for x in network.demands]}
)
n_epochs = 15

for i in range(n_epochs):
    evo.step()

print("\nBest individuals")
print(evo.pool.individuals[0].value)

Evolution initialization
Population initialization
Individual initialization
[[10, 0, 0, 8], [9, 1, 7], [0, 16, 0, 0], [3, 4, 3, 0], [11, 4, 1, 1], [17, 0, 3, 0], [15, 0, 0, 0], [12, 1, 3, 2], [5, 1, 8, 5], [7, 3, 4, 0], [4, 2, 3, 3], [11, 1, 0, 0], [0, 12, 3, 1], [4, 12, 0, 1], [6, 5, 1, 2], [13, 0, 0, 0], [9, 0, 2], [7, 10, 3, 0], [16, 1, 1], [14, 2, 0, 0], [7, 4, 3, 5], [5, 2, 1, 7], [7, 5, 3, 0], [17, 0, 2, 0], [12, 5, 1, 1], [5, 8, 4], [9, 2, 0, 0], [2, 13], [1, 10, 1, 0], [3, 3, 2, 2], [2, 2, 8], [2, 8, 0, 1], [7, 2, 3], [4, 4, 0, 3], [8, 2, 0, 1], [4, 7], [1, 7, 1, 6], [12, 5, 0], [3, 9, 6, 2], [6, 2, 2, 3], [5, 3, 1, 4], [9, 3, 3], [6, 1, 5, 1], [5, 14, 0, 0], [4, 7, 2, 2], [4, 8, 1, 0], [1, 18, 0, 0], [12, 1, 6], [10, 0, 1, 1], [7, 7, 1, 2], [5, 1, 6, 1], [11, 2, 5], [11, 5, 2, 1], [3, 7, 4, 2], [13, 3, 0, 0], [6, 1, 1, 5], [16, 0, 0], [5, 13], [0, 2, 11, 2], [16, 2], [14], [8, 1, 2, 7], [3, 14, 0, 3], [15, 0, 0, 3], [8, 2], [13, 0, 3, 1]]
Individual initialization
[[12, 4, 2,

In [10]:
[x.demand_volume for x in network.demands]

[3, 4, 5, 2, 3, 4]

In [11]:
[x.number_of_paths for x in network.demands]

[3, 3, 2, 3, 3, 3]

In [83]:
init_params = {"seed": 10, "number_of_paths_list": [x.number_of_paths for x in network.demands], "demand_volume_list": [x.demand_volume for x in network.demands]}

In [84]:
for i in range(10):    

    random.seed(init_params["seed"])
    
    flows = []
    
    for number_of_paths, demand_volume in zip(init_params["number_of_paths_list"], init_params["demand_volume_list"]):
        demand_units = []
        demand_limit = copy.copy(demand_volume)

        for i in range(number_of_paths - 1):
            current_units = random.randint(0, demand_limit)

            demand_units.append(current_units)

            demand_limit -= current_units

        demand_units.append(demand_limit)

        flows.append(demand_units)

    print(flows)

[[0, 3, 0], [3, 0, 1], [1, 4], [1, 1, 0], [2, 0, 1], [0, 4, 0]]
[[0, 3, 0], [3, 0, 1], [1, 4], [1, 1, 0], [2, 0, 1], [0, 4, 0]]
[[0, 3, 0], [3, 0, 1], [1, 4], [1, 1, 0], [2, 0, 1], [0, 4, 0]]
[[0, 3, 0], [3, 0, 1], [1, 4], [1, 1, 0], [2, 0, 1], [0, 4, 0]]
[[0, 3, 0], [3, 0, 1], [1, 4], [1, 1, 0], [2, 0, 1], [0, 4, 0]]
[[0, 3, 0], [3, 0, 1], [1, 4], [1, 1, 0], [2, 0, 1], [0, 4, 0]]
[[0, 3, 0], [3, 0, 1], [1, 4], [1, 1, 0], [2, 0, 1], [0, 4, 0]]
[[0, 3, 0], [3, 0, 1], [1, 4], [1, 1, 0], [2, 0, 1], [0, 4, 0]]
[[0, 3, 0], [3, 0, 1], [1, 4], [1, 1, 0], [2, 0, 1], [0, 4, 0]]
[[0, 3, 0], [3, 0, 1], [1, 4], [1, 1, 0], [2, 0, 1], [0, 4, 0]]


In [52]:
[i for i, e in enumerate([1,2,0,0,3,0,2]) if e != 0]

[0, 1, 4, 6]

In [94]:
mutate_params = {"seed": 55, "number_of_demands": network.number_of_demands, "mutation_probability": 0.25}

In [12]:
for demand in range(0, network.number_of_demands):
    for path in network.demands[demand].paths:
        print(path)

Path(demand_path_id=1, links=[1])
Path(demand_path_id=2, links=[2, 3])
Path(demand_path_id=3, links=[2, 5, 4])
Path(demand_path_id=1, links=[2])
Path(demand_path_id=2, links=[1, 3])
Path(demand_path_id=3, links=[1, 4, 5])
Path(demand_path_id=1, links=[1, 4])
Path(demand_path_id=2, links=[2, 5])
Path(demand_path_id=1, links=[3])
Path(demand_path_id=2, links=[1, 2])
Path(demand_path_id=3, links=[4, 5])
Path(demand_path_id=1, links=[4])
Path(demand_path_id=2, links=[3, 5])
Path(demand_path_id=3, links=[1, 2, 5])
Path(demand_path_id=1, links=[5])
Path(demand_path_id=2, links=[3, 4])
Path(demand_path_id=3, links=[2, 1, 4])


In [15]:
def fitness(solution):
    flows_allocation = 0
    
    for link in range(1, network.number_of_links + 1):
        link_load = 0
        for demand in range(0, network.number_of_demands):
            for path in network.demands[demand].paths:
                if link in path.links:
                    link_load += solution[demand][path.demand_path_id - 1]
        link_size = link_load / network.links[link - 1].link_module
        flows_allocation += network.links[link - 1].module_cost * link_size
    
    return flows_allocation     

solution = [[0, 3, 0], [2, 0, 2], [2, 3], [1, 0, 1], [1, 2, 0], [2, 2, 0]]

fitness(solution)

19.0